In [1]:
#import libraries
import pandas as pd
import lyricsgenius as genius #used to interface with Genius API
import string
import re

In [2]:
#token provided by Genius API
%store -r client_access_token

In [3]:
#initiate Genius
genius = genius.Genius(client_access_token)

In [4]:
def get_lyrics(track,artist):
    '''
    function returns song's lyrics
    parameters:
        track-->str
        artist-->str
    '''
    
    return genius.search_song(track,artist).lyrics

In [5]:
def get_df_songs(track_list,artist_list):
    '''
    function obtains lyrics and returns dataframe with columns for track, artist, lyrics
    parameters:
        track_list-->list of str 
        artist_list-->list of str
    '''
    
    #get lyrics for each song
    lyrics_list = [get_lyrics(track_list[x],artist_list[x]) for x in range(len(track_list))]
    
    return pd.DataFrame(data={'track':track_list,'artist':artist_list,'lyrics':lyrics_list})

In [6]:
def clean_lyrics(df,col):
    '''
    function cleans text (song lyrics), formats specified column, and returns cleaned dataframe
    parameters:
        df-->pandas dataframe
        col-->column to clean (str)
    '''
    
    df = df
    df[col] = df[col].str.lower() #make all text lowercase
    df[col] = df[col].str.replace(r'\n',' ',regex=True) #replace '\n' character with space
    df[col] = df[col].str.replace(r'\[[^\[\]]*]','',regex=True) #remove brackets and inside text
    df[col] = df[col].str.replace(r'[^\w\d\'\s]+','',regex=True) #remove extra characters
    df[col] = df[col].str.strip() #remove extra whitespace

    return df

In [7]:
#pull lyrics for songs in question
#need track_list and artist_list from Spotify search
track_list = ['Shallow','Thriller','Lose Yourself','Before He Cheats','Waterfalls'] #sample songs
artist_list = ['Lady Gaga','Michael Jackson','Eminem','Carrie Underwood','TLC'] #sample songs

In [8]:
#create dataframe
df = get_df_songs(track_list,artist_list)

Searching for "Shallow" by Lady Gaga...
Done.
Searching for "Thriller" by Michael Jackson...
Done.
Searching for "Lose Yourself" by Eminem...
Done.
Searching for "Before He Cheats" by Carrie Underwood...
Done.
Searching for "Waterfalls" by TLC...
Done.


In [9]:
df

,track,artist,lyrics
0,Shallow,Lady Gaga,"[Verse 1: Bradley Cooper]\nTell me somethin', ..."
1,Thriller,Michael Jackson,[Verse 1: Michael Jackson]\nIt's close to midn...
2,Lose Yourself,Eminem,"[Intro]\nLook, if you had one shot or one oppo..."
3,Before He Cheats,Carrie Underwood,"[Verse 1]\nRight now, he's probably slow danci..."
4,Waterfalls,TLC,[Verse 1: T-Boz]\nA lonely mother gazing out o...


In [10]:
df_clean = clean_lyrics(df,'lyrics')

In [11]:
df_clean

,track,artist,lyrics
0,Shallow,Lady Gaga,tell me somethin' girl are you happy in this m...
1,Thriller,Michael Jackson,it's close to midnight something evil's lurkin...
2,Lose Yourself,Eminem,look if you had one shot or one opportunity to...
3,Before He Cheats,Carrie Underwood,right now he's probably slow dancing with a bl...
4,Waterfalls,TLC,a lonely mother gazing out of the window stari...


In [12]:
df.iloc[1].lyrics

"it's close to midnight something evil's lurking in the dark under the moonlight you see a sight that almost stops your heart you try to scream but terror takes the sound before you make it you start to freeze as horror looks you right between your eyes you're paralyzed   'cause this is thriller thriller night and no one's gonna save you from the beast about to strike you know it's thriller thriller night you're fighting for your life inside a killer thriller tonight yeah ooh   you hear the door slam and realize there's nowhere left to run you feel the cold hand and wonder if you'll ever see the sun you close your eyes and hope that this is just imagination girl but all the while you hear a creature creeping up behind you're out of time  'cause this is thriller thriller night there ain't no second chance against the thing with 40 eyes girl ooh ooh thriller thriller night you're fighting for your life inside a killer thriller tonight   night creatures call and the dead start to walk in 